In [1]:
%load_ext lab_black
%load_ext autotime
import pandas as pd
import numpy as np

time: 274 ms (started: 2022-09-18 16:40:24 -07:00)


This notebook relies heavily on various other notebooks that know what they are doing from the biological side of things to a far greater extent than me:

* [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb) -- this one in particular
* [BerensLab notebook](https://github.com/berenslab/umi-normalization/blob/main/07_compute_retina.ipynb)
* [hemberg-lab shell script](https://github.com/hemberg-lab/scRNA.seq.datasets/blob/8fd6ec9b85917e3f4a0c097006083e9c9e62cde8/bash/macosko.sh)

**Warning**: we are dealing with a big file here. While it's only 51 MB to download, it's 2.3 GB when unzipped and while it has a mere 25000 rows (why, that's not even *half* an MNIST!), it has nearly 50000 columns. So reading this in takes a *long* time. Make sure you have the time (and RAM) to do this: The final dataframe takes up nearly 10 GB of RAM and there is a spike of up to nearly 12 GB just before it finishes.

## Reading in the data the hard way

To help with handling the memory requirements, we are only going to use the columns which are listed in this file of cluster identities:

In [2]:
cluster_ids = pd.read_csv(
    "https://scrnaseq-public-datasets.s3.amazonaws.com/manual-data/macosko/retina_clusteridentities.txt",
    header=None,
    index_col=0,
    sep="\t",
).squeeze(1)

time: 1.64 s (started: 2022-09-18 16:40:24 -07:00)


In [3]:
cluster_ids.index

Index(['r1_GGCCGCAGTCCG', 'r1_CTTGTGCGGGAA', 'r1_GCGCAACTGCTC',
       'r1_GATTGGGAGGCA', 'r1_GTGCCGCCTCTC', 'r1_CCTGTGACACAC',
       'r1_AATCTCGTTAAT', 'r1_GATTTCCTCTGA', 'r1_GAAGGCTGGAAC',
       'r1_TTCCGGCTGTAC',
       ...
       'p1_TGTACACCTGGG', 'p1_GAGGGGCTCTAA', 'p1_AGCCAAGGCTCG',
       'p1_TGAGTCGTCTTA', 'p1_CGAATACGTGTC', 'p1_TCAAAAGCCGGG',
       'p1_ATTAAGTTCCAA', 'p1_CTGTCTGAGACC', 'p1_TAACGCGCTCCT',
       'p1_ATTCTTGTTCTT'],
      dtype='object', name=0, length=44808)

time: 7.22 ms (started: 2022-09-18 16:40:26 -07:00)


There are 44808 cells, which is a lot of columns but we can still avoid reading nearly 5000 columns worth of data. Also, the largest count in the columns we care about is only `671` (it's `679` for the entire dataset but we won't use that column), which means we can use `np.int16` for the `dtype` of the dataframe, rather than the typical `np.int64`. This should save a factor of 4 in each column's storage (that can be checked by looking at the output of `data.memory_usage(index=False, deep=True)` if you so wish).

The downside is that we can't read the string-based index in at the same time (we'll do that by reading in separately), but doing things this way results in the RAM usage being below 3GB. Also, we need to read in `cluster_ids` anyway for our `target` (label) data, so we are just making it work for us early.

In [4]:
data = pd.read_csv(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63472&format=file&file=GSE63472%5FP14Retina%5Fmerged%5Fdigital%5Fexpression%2Etxt%2Egz",
    sep="\t",
    compression="gzip",
    header=0,
    chunksize=400,
    dtype=np.int16,
    usecols=lambda x: x in cluster_ids.index,
)

time: 13.1 s (started: 2022-09-18 16:40:26 -07:00)


Still here? Yeah, well that's because of the use of `chunksize` nothing really happens until you try to create the complete dataframe, which we will do below. Good luck everyone...

In [5]:
data = pd.concat(data)

time: 10min 12s (started: 2022-09-18 16:40:39 -07:00)


This can take quite a while (e.g. over 10 minutes for me). If your kernel didn't survive that you may have to fiddle with the `chunksize`. Once that works we need the index column which is column `0`:

In [6]:
data_index = pd.read_csv(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63472&format=file&file=GSE63472%5FP14Retina%5Fmerged%5Fdigital%5Fexpression%2Etxt%2Egz",
    sep="\t",
    compression="gzip",
    header=0,
    index_col=0,
    chunksize=400,
    usecols=[0],
)
data_index = pd.concat(data_index)

time: 36.9 s (started: 2022-09-18 16:50:51 -07:00)


And now add the index to `data`:

In [7]:
data.set_index(data_index.index, inplace=True)

time: 1.25 ms (started: 2022-09-18 16:51:28 -07:00)


This process is a bit fiddly and slow (although not a lot slower than the more straightforward approach), but it did mean my kernel did not exceed ~2.6GB of RAM usage.

## Reading in the data the easy (but RAM-consuming) way

If you have lots of RAM and all of the above seems a real pain, then you can just do the following, but I have commented it out, because I would hate to run this by accident:

In [8]:
# NOT a good idea unless you have lots of RAM
# data = pd.read_csv(
#     "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63472&format=file&file=GSE63472%5FP14Retina%5Fmerged%5Fdigital%5Fexpression%2Etxt%2Egz",
#     sep="\t",
#     compression="gzip",
#     header=0,
#     index_col=0,
# )
# remove the columns we don't need
# data = data.loc[:, data.columns.isin(cluster_ids.index)]

time: 3.83 ms (started: 2022-09-18 16:51:28 -07:00)


## Save the intermediate data

However you do it, just to be on the safe side you might want to consider pickling that file to disk before moving on (gzipping will reduce the size of the `np.int64` version of the output file from 10 GB to 70 MB but can take a few minutes to do its work -- the `np.int16` version is around 50 MB compressed):

In [9]:
# import gzip
# import pickle
# from pathlib import Path

# path = "path-to-wherever-you-save-this-stuff"

# uncomment this if you just want the pickle
# with open(
#     Path(path) / "GSE63472_P14Retina_merged_digital_expression.pkl", "wb"
# ) as f:
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

# uncomment this if you want the pickle gzipped too
# with gzip.open(
#     Path(path) / "GSE63472_P14Retina_merged_digital_expression.pkl.gz", "wb"
# ) as f:
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

time: 3.37 ms (started: 2022-09-18 16:51:28 -07:00)


## Load the intermediate data

If something goes wrong, get the data back. If you aren't using the `np.int16` version of the data, be aware that although this is several times faster than writing, you may see a memory spike (in my case, up to 16 GB) just before the data is returned:

In [10]:
# import gzip
# import pickle
# from pathlib import Path

# path = "path-to-wherever-you-save-this-stuff"

# uncomment this if you wrote the pickle file
# with open(
#     Path(path) / "GSE63472_P14Retina_merged_digital_expression.pkl", "rb",
# ) as f:
#     data = pickle.load(f)

# uncomment this if you wrote the gzipped pickle file
# with gzip.open(
#     Path(path) / "GSE63472_P14Retina_merged_digital_expression.pkl", "rb",
# ) as f:
#     data = pickle.load(f)

time: 2.52 ms (started: 2022-09-18 16:51:28 -07:00)


Also remember to load `cluster_ids` back in (near the top of this document).

In [11]:
data.shape

(24658, 44808)

time: 10 ms (started: 2022-09-18 16:51:28 -07:00)


## Name the clusters by cell type

Back to the clusters for a bit (this bit is taken from  [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb)).

In [12]:
cluster_ids.head()

0
r1_GGCCGCAGTCCG     2
r1_CTTGTGCGGGAA     2
r1_GCGCAACTGCTC     2
r1_GATTGGGAGGCA     2
r1_GTGCCGCCTCTC    25
Name: 1, dtype: int64

time: 8.06 ms (started: 2022-09-18 16:51:28 -07:00)


Reorder `cluster_ids` so the contents are in the same order as the columns of `data`:

In [13]:
cluster_ids = cluster_ids[data.columns.values]

time: 23 ms (started: 2022-09-18 16:51:28 -07:00)


In [14]:
cluster_ids.head()

0
r1_GGCCGCAGTCCG     2
r1_CTTGTGCGGGAA     2
r1_GCGCAACTGCTC     2
r1_GATTGGGAGGCA     2
r1_GTGCCGCCTCTC    25
Name: 1, dtype: int64

time: 3.34 ms (started: 2022-09-18 16:51:28 -07:00)


Same order as the dataframe:

In [15]:
data.columns.values[:5]

array(['r1_GGCCGCAGTCCG', 'r1_CTTGTGCGGGAA', 'r1_GCGCAACTGCTC',
       'r1_GATTGGGAGGCA', 'r1_GTGCCGCCTCTC'], dtype=object)

time: 2.95 ms (started: 2022-09-18 16:51:28 -07:00)


There should also not be cells with missing data:

In [16]:
cluster_ids.isna().any()

False

time: 3.59 ms (started: 2022-09-18 16:51:28 -07:00)


The cluster ids are all numeric:

In [17]:
cluster_ids.value_counts().head()

24    29400
26     2217
25     1868
34     1624
33      849
Name: 1, dtype: int64

time: 4.52 ms (started: 2022-09-18 16:51:28 -07:00)


But they do correspond to a cell type:

In [18]:
cell_types = cluster_ids.astype(object)

cell_types.loc[cell_types == 1] = "Horizontal cells"
cell_types.loc[cell_types == 2] = "Retinal ganglion cells"
cell_types.loc[cell_types.isin(range(3, 24))] = "Amacrine cells"
cell_types.loc[cell_types == 24] = "Rods"
cell_types.loc[cell_types == 25] = "Cones"
cell_types.loc[cell_types.isin(range(26, 34))] = "Bipolar cells"
cell_types.loc[cell_types == 34] = "Muller glia"
cell_types.loc[cell_types == 35] = "Astrocytes"
cell_types.loc[cell_types == 36] = "Fibroblasts"
cell_types.loc[cell_types == 37] = "Vascular endothelium"
cell_types.loc[cell_types == 38] = "Pericytes"
cell_types.loc[cell_types == 39] = "Microglia"

cell_types.value_counts().head()

Rods              29400
Bipolar cells      6285
Amacrine cells     4426
Cones              1868
Muller glia        1624
Name: 1, dtype: int64

time: 36.9 ms (started: 2022-09-18 16:51:28 -07:00)


## Gene selection

Requires converting `data` to sparse format. All the functions here are from the [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb) (lightly modified to only include the needed functionality).

In [19]:
import scipy.sparse as sp

counts = sp.csr_matrix(data.values)
counts

<24658x44808 sparse matrix of type '<class 'numpy.int16'>'
	with 32805477 stored elements in Compressed Sparse Row format>

time: 8.16 s (started: 2022-09-18 16:51:28 -07:00)


In [20]:
def select_genes(
    data,
    n,
    threshold=0,
    atleast=10,
    yoffset=0.02,
    xoffset=5,
    decay=1,
):
    zeroRate = 1 - np.squeeze(np.array((data > threshold).mean(axis=0)))
    A = data.multiply(data > threshold)
    A.data = np.log2(A.data)
    meanExpr = np.zeros_like(zeroRate) * np.nan
    detected = zeroRate < 1
    meanExpr[detected] = np.squeeze(np.array(A[:, detected].mean(axis=0))) / (
        1 - zeroRate[detected]
    )

    lowDetection = np.array(np.sum(data > threshold, axis=0)).squeeze() < atleast
    zeroRate[lowDetection] = np.nan
    meanExpr[lowDetection] = np.nan

    up = 10
    low = 0
    for _ in range(100):
        nonan = ~np.isnan(zeroRate)
        selected = np.zeros_like(zeroRate).astype(bool)
        selected[nonan] = (
            zeroRate[nonan] > np.exp(-decay * (meanExpr[nonan] - xoffset)) + yoffset
        )
        if np.sum(selected) == n:
            break
        if np.sum(selected) < n:
            up = xoffset
            xoffset = (xoffset + low) / 2
        else:
            low = xoffset
            xoffset = (xoffset + up) / 2
    print(f"Chosen offset: {xoffset:.2f}")

    return selected

time: 9.78 ms (started: 2022-09-18 16:51:36 -07:00)


In [21]:
gene_mask = select_genes(counts.T, n=3000)

Chosen offset: 0.19
time: 905 ms (started: 2022-09-18 16:51:36 -07:00)


## Create the final log-cpm data

Because of the very skewed nature of the count data, the final form of the data is stored as the (one plus the) log of the counts per million (CPM).

In [22]:
def calculate_cpm(x):
    """Calculate counts-per-million on data where the rows are genes."""
    normalization = np.sum(x, axis=0)
    # On sparse matrices, the sum will be 2d. We want a 1d array
    normalization = np.squeeze(np.asarray(normalization))
    # Straight up division is not an option since this will form a full dense
    # matrix if `x` is sparse. Divison can be expressed as the dot product with
    # a reciprocal diagonal matrix
    normalization = sp.diags(1 / normalization, offsets=0)
    cpm_counts = np.dot(x, normalization)

    return cpm_counts * 1e6

time: 2.2 ms (started: 2022-09-18 16:51:37 -07:00)


In [23]:
cpm_counts = calculate_cpm(counts)
cpm_counts

<24658x44808 sparse matrix of type '<class 'numpy.float64'>'
	with 32805477 stored elements in Compressed Sparse Row format>

time: 554 ms (started: 2022-09-18 16:51:37 -07:00)


In [24]:
def log_normalize(data):
    """Perform log transform log(x + 1)"""
    data = data.copy()
    data.data = np.log2(data.data + 1)
    return data

time: 1.54 ms (started: 2022-09-18 16:51:38 -07:00)


In [25]:
log_counts = log_normalize(cpm_counts)
log_counts

<24658x44808 sparse matrix of type '<class 'numpy.float64'>'
	with 32805477 stored elements in Compressed Sparse Row format>

time: 360 ms (started: 2022-09-18 16:51:38 -07:00)


Then the log cpm data is transposed, so the cells are rows and the genes are columns and only the 3000 selected genes are kept:

In [26]:
x = log_counts.T[:, gene_mask].toarray()
x.shape

(44808, 3000)

time: 633 ms (started: 2022-09-18 16:51:38 -07:00)


In [27]:
x.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

time: 2.53 ms (started: 2022-09-18 16:51:39 -07:00)


In [28]:
target = pd.DataFrame(dict(ClusterId=cluster_ids, CellType=cell_types))

time: 3.28 ms (started: 2022-09-18 16:51:39 -07:00)


While we're at it, let's get rid of that `0` as the index name:

In [29]:
target.index.name = None

time: 1.78 ms (started: 2022-09-18 16:51:39 -07:00)


## Data Pipeline

Also we will include a palette used for plotting colors when using the cell types as a legend (once again from openTSNE):

In [30]:
target_palette = dict(
    CellType={
        "Amacrine cells": "#A5C93D",
        "Astrocytes": "#8B006B",
        "Bipolar cells": "#2000D7",
        "Cones": "#538CBA",
        "Fibroblasts": "#8B006B",
        "Horizontal cells": "#B33B19",
        "Microglia": "#8B006B",
        "Muller glia": "#8B006B",
        "Pericytes": "#8B006B",
        "Retinal ganglion cells": "#C38A1F",
        "Rods": "#538CBA",
        "Vascular endothelium": "#8B006B",
    }
)

time: 4.58 ms (started: 2022-09-18 16:51:39 -07:00)


In [31]:
from drnb.io.pipeline import create_default_pipeline

data_result = create_default_pipeline(check_for_duplicates=True).run(
    "macosko2015",
    data=x,
    target=target,
    target_palette=target_palette,
    tags=["highdim", "scRNAseq"],
    url="https://doi.org/10.1016/j.cell.2015.05.002",
    verbose=True,
)

[16:51:43] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=621894;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=258300;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#392\392]8;;\

           INFO     Initial data shape: (44808, 3000)                                                ]8;id=417283;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=126919;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#80\80]8;;\

           INFO     Keeping all columns                                                            ]8;id=228995;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=32698;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     Data shape after filtering columns: (44808, 3000)                               ]8;id=554559;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=833476;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#151\151]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=831810;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=465016;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#135\135]8;;\

...

           INFO     Writing csv format to triplets/macosko2015.5.1337.l2.csv                        ]8;id=348088;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=170911;file:///home/james/dev/drnb/src/drnb/io/__init__.py#220\220]8;;\

[16:53:21] INFO     Writing numpy format to triplets/macosko2015.5.1337.idx.npy                     ]8;id=46604;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=134823;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing numpy format to triplets/macosko2015.5.1337.l2.npy                      ]8;id=185495;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=612052;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing pipeline result for macosko2015                                         ]8;id=300369;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=205923;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#128\128]8;;\

time: 1min 42s (started: 2022-09-18 16:51:39 -07:00)


The openTSNE notebook standardizes the data, so here's a pipeline that does that too (via `scale="z"`):

In [32]:
data_result = create_default_pipeline(scale="z").run(
    "macosko2015z",
    data=x,
    target=target,
    target_palette=target_palette,
    tags=["highdim", "scRNAseq"],
    url="https://doi.org/10.1016/j.cell.2015.05.002",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=349643;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=644004;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#392\392]8;;\

           INFO     Initial data shape: (44808, 3000)                                                ]8;id=197900;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=589259;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#80\80]8;;\

           INFO     Keeping all columns                                                            ]8;id=312409;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=136076;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     Data shape after filtering columns: (44808, 3000)                               ]8;id=39401;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=581314;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#151\151]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=169219;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=737050;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#135\135]8;;\

...

[16:55:32] INFO     Writing csv format to triplets/macosko2015z.5.1337.l2.csv                       ]8;id=275701;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=184322;file:///home/james/dev/drnb/src/drnb/io/__init__.py#220\220]8;;\

           INFO     Writing numpy format to triplets/macosko2015z.5.1337.idx.npy                    ]8;id=112190;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=497497;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing numpy format to triplets/macosko2015z.5.1337.l2.npy                     ]8;id=168134;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=759618;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing pipeline result for macosko2015z                                        ]8;id=601779;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=940507;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#128\128]8;;\

time: 2min 11s (started: 2022-09-18 16:53:21 -07:00)


### Standardize and PCA=50

To be as close as possible to the procedure in the [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb), after standardizing PCA is used to reducing the initial dimensionality down to 50 dimensions. Here is that as a pipeline: 

In [33]:
data_result = create_default_pipeline(scale="z", reduce=50).run(
    "macosko2015z-pca50",
    data=x,
    target=target,
    target_palette=target_palette,
    tags=["scRNAseq"],
    url="https://doi.org/10.1016/j.cell.2015.05.002",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=783106;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=720443;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#392\392]8;;\

           INFO     Initial data shape: (44808, 3000)                                                ]8;id=135090;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=797995;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#80\80]8;;\

           INFO     Keeping all columns                                                            ]8;id=743008;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=829713;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     Data shape after filtering columns: (44808, 3000)                               ]8;id=782228;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=33774;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#151\151]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=205748;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=659174;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#135\135]8;;\

...

[16:56:16] INFO     Writing csv format to triplets/macosko2015z-pca50.5.1337.l2.csv                 ]8;id=295891;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=517425;file:///home/james/dev/drnb/src/drnb/io/__init__.py#220\220]8;;\

[16:56:17] INFO     Writing numpy format to triplets/macosko2015z-pca50.5.1337.idx.npy              ]8;id=362784;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=306166;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing numpy format to triplets/macosko2015z-pca50.5.1337.l2.npy               ]8;id=347662;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=591187;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing pipeline result for macosko2015z-pca50                                  ]8;id=510360;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=607405;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#128\128]8;;\

time: 44.4 s (started: 2022-09-18 16:55:32 -07:00)


### Just PCA

This one is all me. I am very interested in what PCA does to this dataset, but I don't want to deal with the extra complication from Z-scaling.

In [34]:
data_result = create_default_pipeline(reduce=50).run(
    "macosko2015-pca50",
    data=x,
    target=target,
    target_palette=target_palette,
    tags=["scRNAseq"],
    url="https://doi.org/10.1016/j.cell.2015.05.002",
    verbose=True,
)

           INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=981277;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=608886;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#392\392]8;;\

           INFO     Initial data shape: (44808, 3000)                                                ]8;id=675166;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=243562;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#80\80]8;;\

           INFO     Keeping all columns                                                            ]8;id=748277;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=455785;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     Data shape after filtering columns: (44808, 3000)                               ]8;id=575253;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=309331;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#151\151]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=708153;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=706955;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#135\135]8;;\

...

[16:57:02] INFO     Writing csv format to triplets/macosko2015-pca50.5.1337.l2.csv                  ]8;id=441725;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=652848;file:///home/james/dev/drnb/src/drnb/io/__init__.py#220\220]8;;\

[16:57:03] INFO     Writing numpy format to triplets/macosko2015-pca50.5.1337.idx.npy               ]8;id=1485;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=488375;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing numpy format to triplets/macosko2015-pca50.5.1337.l2.npy                ]8;id=714593;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=306167;file:///home/james/dev/drnb/src/drnb/io/__init__.py#243\243]8;;\

           INFO     Writing pipeline result for macosko2015-pca50                                   ]8;id=369339;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=948985;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#128\128]8;;\

time: 45.9 s (started: 2022-09-18 16:56:17 -07:00)
